<a href="https://colab.research.google.com/github/Udinkosd/Data-Mining-2024/blob/main/Check_Point_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediksi Gaji Mahasiswa

#**IMPORT LIBRARY**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mtick
import matplotlib.gridspec as gridspec
import plotly.express as px
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.model_selection import KFold

# **DATA PREPARATION**

##**DATA CLEANING**

###**Penanganan Missing Values**

#####**Pengecekkan Missing Values**

In [30]:
url = 'https://raw.githubusercontent.com/Udinkosd/Data-Mining-2024/main/Salary%20Dataset.csv'
df = pd.read_csv(url)

print((df.isna().sum() / len(df)) * 100)

Age                    0.533333
Gender                 0.533333
Education Level        0.533333
Job Title              0.533333
Years of Experience    0.533333
Salary                 0.533333
dtype: float64


In [31]:
df[df['Age'].isnull()]
df[df['Gender'].isnull()]
df[df['Education Level'].isnull()]

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
172,NaN,NaN,NaN,NaN,NaN,NaN
260,NaN,NaN,NaN,NaN,NaN,NaN


Dalam pengecekan tersebut, ditemukan missing value sebesar 53.33% atau kurang dari 70%, maka langkah yang diambil adalah melakukan imputasi terhadap missing value tersebut. Adapun langkahnya adalah sebagai berikut ini.

#####**Imputasi Missing Values**

In [32]:
mode_education = df['Education Level'].mode()[0]
df['Education Level'].fillna(mode_education, inplace=True)

In [33]:
mean_age = df['Age'].mean()
df['Age'].fillna(mean_age, inplace=True)

mean_experience = df['Years of Experience'].mean()
df['Years of Experience'].fillna(mean_experience, inplace=True)

mean_salary = df['Salary'].mean()
df['Salary'].fillna(mean_salary, inplace=True)

Disini kita akan melakukan imputasi missing value dengan menggunakan mean / nilai rata-rata, yaitu:

Mode (Modus) untuk 'Education Level': Digunakan untuk mengisi nilai yang hilang dalam kolom 'Education Level'. Yaitu dengan mencari nilai modus pada kolom Education Level.

Mean (Rata-rata) untuk 'Age', 'Years of Experience', dan 'Salary': Digunakan untuk mengisi nilai yang hilang dalam kolom 'Age', 'Years of Experience', dan 'Salary'. Rata-rata adalah jumlah dari semua nilai dalam suatu kolom dibagi dengan jumlah total nilai. Cara ini adalah pendekatan umum untuk mengisi nilai yang hilang ketika data terdistribusi secara normal.

#####**Pengecekkan Ulang**

In [34]:
pd.DataFrame(df.isna().sum() / len(df) * 100, columns=['Null Ratio %'])

,Null Ratio %
Age,0.000000
Gender,0.533333
Education Level,0.000000
Job Title,0.533333
Years of Experience,0.000000
Salary,0.000000


Output tersebut menunjukkan persentase nilai yang hilang (missing values) dalam setiap kolom DataFrame, diukur sebagai persentase dari total jumlah baris DataFrame.

Age: Tidak ada nilai yang hilang (0.0000%)
Gender dan Job Title: Masing-masing memiliki 0.5333% nilai yang hilang.
Education Level, Years of Experience, dan Salary: Tidak ada nilai yang hilang (0.0000%).

secara umum, persentase nilai yang hilang sekitar 0.5333% masih dapat diterima. Meskipun keputusan akhir kembali kepada tujuan penelitian data.

###**Penanganan Duplicated Values**

#####**Pengecekkan Nilai Duplikat**

In [35]:
duplicate_rows = df[df.duplicated()]
print("Duplicate Rows except first occurrence based on all columns:")
print(duplicate_rows)

Duplicate Rows except first occurrence based on all columns:
           Age  Gender Education Level                              Job Title  \
195  28.000000    Male      Bachelor's                Junior Business Analyst   
250  30.000000  Female      Bachelor's           Junior Marketing Coordinator   
251  38.000000    Male        Master's                   Senior IT Consultant   
252  45.000000  Female             PhD                Senior Product Designer   
253  28.000000    Male      Bachelor's  Junior Business Development Associate   
254  35.000000  Female      Bachelor's               Senior Marketing Analyst   
255  44.000000    Male      Bachelor's               Senior Software Engineer   
256  34.000000  Female        Master's               Senior Financial Advisor   
257  35.000000    Male      Bachelor's             Senior Project Coordinator   
258  50.000000  Female             PhD                 Director of Operations   
260  37.431635     NaN      Bachelor's          

Berdasarkan hasil pengecekan nilai duplikat dalam dataset, ditemukan bahwa beberapa baris memiliki nilai yang sama untuk setiap kolom, khususnya pada atribut yang berkaitan langsung dengan analisis gaji seperti 'Job Title', 'Years of Experience', dan 'Salary'. Dalam konteks dataset yang menunjukkan informasi gaji para lulusan, adanya duplikasi data seperti ini diharapkan dan tidak dianggap sebagai masalah yang mengganggu analisis. Sehingga langkah ini dapat dilewati.

###**Penanganan Outliers**

#####**Pengecekkan Outliers**

In [36]:
results = []

cols = df.select_dtypes(include=['float64', 'int64'])

for col in cols:
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5*iqr
  upper_bound = q3 + 1.5*iqr
  outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
  percent_outliers = (len(outliers)/len(df))*100
  results.append({'Kolom': col, 'Persentase Outliers': percent_outliers})

# Dataframe dari list hasil
results_df = pd.DataFrame(results)
results_df.set_index('Kolom', inplace=True)
results_df = results_df.rename_axis(None, axis=0).rename_axis('Kolom', axis=1)

# Tampilkan dataframe
display(results_df)

Kolom,Persentase Outliers
Age,0.0
Years of Experience,0.0
Salary,0.0


Dalam dataset tersebut, tidak ditemukan outlier values dalam kolom Age, Years of Experience, dan Salary. Ini menunjukkan bahwa tidak ada nilai ekstrim atau tidak biasa dalam data untuk ketiga kolom tersebut. Sehingga langkah ini dapat dilewati.

##**CONSTRUCT DATA**

###**Melakukan Rekayasa Fitur (Feature Engineering)**

In [37]:
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


In [38]:
# Melihat seluruh Job Title
job_titles = df['Job Title'].unique()
print("Seluruh Job Titles:")
print(job_titles)

# Menghitung jumlah duplikat untuk setiap Job Title
duplicate_counts = df['Job Title'].value_counts()
print("\nJumlah duplikat untuk setiap Job Title:")
print(duplicate_counts)

Seluruh Job Titles:
['Software Engineer' 'Data Analyst' 'Senior Manager' 'Sales Associate'
 'Director' 'Marketing Analyst' 'Product Manager' 'Sales Manager'
 'Marketing Coordinator' 'Senior Scientist' 'Software Developer'
 'HR Manager' 'Financial Analyst' 'Project Manager' 'Customer Service Rep'
 'Operations Manager' 'Marketing Manager' 'Senior Engineer'
 'Data Entry Clerk' 'Sales Director' 'Business Analyst' 'VP of Operations'
 'IT Support' 'Recruiter' 'Financial Manager' 'Social Media Specialist'
 'Software Manager' 'Junior Developer' 'Senior Consultant'
 'Product Designer' 'CEO' 'Accountant' 'Data Scientist'
 'Marketing Specialist' 'Technical Writer' 'HR Generalist'
 'Project Engineer' 'Customer Success Rep' 'Sales Executive' 'UX Designer'
 'Operations Director' 'Network Engineer' 'Administrative Assistant'
 'Strategy Consultant' 'Copywriter' 'Account Manager'
 'Director of Marketing' 'Help Desk Analyst' 'Customer Service Manager'
 'Business Intelligence Analyst' 'Event Coordinator'

In [39]:
def categorize_job_title(title):
    engineering_titles = ['Software Engineer', 'Data Analyst', 'Software Developer', 'IT Support', 'Network Engineer', 'Web Developer']
    management_titles = ['Senior Manager', 'Director', 'Product Manager', 'Operations Manager', 'Project Manager', 'Business Analyst', 'Business Development Manager', 'Marketing Manager']
    finance_titles = ['Financial Analyst', 'Accountant', 'Financial Manager', 'Chief Financial Officer (CFO)']
    marketing_titles = ['Marketing Coordinator', 'Social Media Specialist', 'Graphic Designer', 'Copywriter', 'Content Marketing Manager']
    hr_customer_service_titles = ['HR Manager', 'Recruiter', 'Customer Service Representative', 'Customer Service Manager', 'Customer Success Manager']

    if title in engineering_titles:
        return 'Engineering'
    elif title in management_titles:
        return 'Management'
    elif title in finance_titles:
        return 'Finance'
    elif title in marketing_titles:
        return 'Marketing'
    elif title in hr_customer_service_titles:
        return 'HR & Customer Service'
    else:
        return 'Other'

df['JobCategory'] = df['Job Title'].apply(categorize_job_title)

Di sini, kita akan mengelompokkan 'Job Title' / pekerjaan ke dalam 6 jenis, yaitu:

1. Engineering & IT: Termasuk Software Engineer, Data Analyst, Software 2.Developer, IT Support, Network Engineer, Web Developer, dll.
2. Manajemen dan Bisnis: Meliputi Senior Manager, Director, Product Manager, Operations Manager, Project Manager, Business Analyst, Business Development Manager, Marketing Manager, dll.
3. Keuangan dan Akuntansi: Seperti Financial Analyst, Accountant, Financial
4. Manager, Chief Financial Officer (CFO), dll.
Pemasaran dan Kreativitas: Contohnya Marketing Coordinator, Social Media Specialist, Graphic Designer, Copywriter, Content Marketing Manager, dll.
5. Sumber Daya Manusia (HR) dan Layanan Pelanggan: Termasuk HR Manager, Recruiter, Customer Service Representative, Customer Success Manager, dll.
6. Other jika tidak disebutkan.

In [40]:
def categorize_age(age):
    if age < 18:
        return 'Child'
    elif 18 <= age < 60:
        return 'Adult'
    else:
        return 'Elderly'

df['AgeCategory'] = df['Age'].apply(categorize_age)

Pada tahap ini, kita akan melakukan pengkategorian usia ke dalam tiga kategori: 'Child', 'Adult', dan 'Elderly'. Karena dataset ini memiliki banyak data, pembagian usia ini akan membantu dalam analisis lebih lanjut.

###**Pengecekkan Hasil**

In [41]:
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,JobCategory,AgeCategory
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,Engineering,Adult
1,28.0,Female,Master's,Data Analyst,3.0,65000.0,Engineering,Adult
2,45.0,Male,PhD,Senior Manager,15.0,150000.0,Management,Adult
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,Other,Adult
4,52.0,Male,Master's,Director,20.0,200000.0,Management,Adult


In [42]:
df['AgeCategory'].value_counts()
df['JobCategory'].value_counts()

Other                    337
Management                13
HR & Customer Service      8
Engineering                7
Marketing                  7
Finance                    3
Name: JobCategory, dtype: int64

Hasil ini menunjukkan bahwa tidak terdapat kategori selain 'Adult' dan 'Elderly' yang artinya rata-rata usianya adalah 18 hingga 60 tahun lebih.

##**DATA REDUCTION**

###**Mengurangi Fitur - Fitur Yang Kurang Relevan**

In [43]:
df_corr = df.corr(numeric_only=True)
px.imshow(df_corr)

Pada tahap pengurangan fitur ini, kita dapat melewatinya karena tidak ada fitur yang dianggap tidak relevan. Semua fitur, mulai dari Age, Gender, Education, Job Title, Years of Experience, hingga Salary, dianggap penting dan relevan untuk analisis. Oleh karena itu, tidak perlu melakukan pengurangan fitur. Semua informasi yang tersedia dapat dimanfaatkan sepenuhnya dalam pemodelan atau analisis yang akan dilakukan. Selanjutnya, kita dapat melanjutkan ke tahap berikutnya dalam proses analisis data.

###**Mapping**

In [44]:
df['AgeCategory'] = df['AgeCategory'].map({'Child': 0, 'Adult': 1, 'Elder': 2}).fillna(1).astype(int)

In [45]:
df['JobCategory'] = df['JobCategory'].map({'Engineering': 0, 'Management': 1, 'Finance': 2, 'Marketing': 3, 'HR & Customer Service': 4, 'Other': 5}).fillna(5).astype(int)

In [46]:
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,JobCategory,AgeCategory
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,0,1
1,28.0,Female,Master's,Data Analyst,3.0,65000.0,0,1
2,45.0,Male,PhD,Senior Manager,15.0,150000.0,1,1
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,5,1
4,52.0,Male,Master's,Director,20.0,200000.0,1,1


Ini adalah penambahan kolom baru dalam dataframe yang menunjukkan kategori usia berdasarkan nilai 'Child', 'Adult', dan 'Elder' pada kolom 'AgeCategory', serta kategori pekerjaan berdasarkan berbagai judul pekerjaan pada kolom 'JobCategory'.

Selanjutnya, kolom tersebut dipecah menjadi beberapa kolom biner untuk setiap kategori usia, misalnya 'AgeCategory_1' untuk kategori 'Child', 'AgeCategory_2' untuk kategori 'Adult', dan 'AgeCategory_3' untuk kategori 'Elder'. Kolom-kolom biner ini menunjukkan apakah suatu data berada dalam kategori usia tertentu atau tidak.

Dan Kolom 'JobCategory' juga dipecah menjadi kolom biner untuk setiap kategori pekerjaan, seperti 'JobCategory_0' untuk kategori 'Engineering', 'JobCategory_1' untuk kategori 'Management', dan seterusnya. Hal ini memudahkan dalam analisis data dengan mempertimbangkan peran pekerjaan dalam pemodelan data dan pengambilan keputusan.

##**DATA TRANSFORMATION**

###**Mapping**

In [47]:
# Melakukan one-hot encoding pada kolom 'Age'
df = pd.concat([df, pd.get_dummies(df['AgeCategory'], prefix='AgeCategory')], axis=1)

In [48]:
# Melakukan one-hot encoding pada kolom 'Gender'
df = pd.concat([df, pd.get_dummies(df['Gender'], prefix='Gender')], axis=1)

In [49]:
# Melakukan one-hot encoding pada kolom 'Job'
df = pd.concat([df, pd.get_dummies(df['JobCategory'], prefix='JobCategory')], axis=1)

In [50]:
df

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,JobCategory,AgeCategory,AgeCategory_1,Gender_Female,Gender_Male,JobCategory_0,JobCategory_1,JobCategory_2,JobCategory_3,JobCategory_4,JobCategory_5
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,0,1,1,0,1,1,0,0,0,0,0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0,0,1,1,1,0,1,0,0,0,0,0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0,1,1,1,0,1,0,1,0,0,0,0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,5,1,1,1,0,0,0,0,0,0,1
4,52.0,Male,Master's,Director,20.0,200000.0,1,1,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,35.0,Female,Bachelor's,Senior Marketing Analyst,8.0,85000.0,5,1,1,1,0,0,0,0,0,0,1
371,43.0,Male,Master's,Director of Operations,19.0,170000.0,5,1,1,0,1,0,0,0,0,0,1
372,29.0,Female,Bachelor's,Junior Project Manager,2.0,40000.0,5,1,1,1,0,0,0,0,0,0,1
373,34.0,Male,Bachelor's,Senior Operations Coordinator,7.0,90000.0,5,1,1,0,1,0,0,0,0,0,1


Pada tahap ini, dilakukan proses transformasi data menggunakan one-hot encoding untuk dua kolom, yaitu 'AgeCategory' dan 'Gender'.

One-hot encoding pada kolom 'AgeCategory':

Dengan menggunakan pd.get_dummies, kolom 'AgeCategory' yang berisi kategori usia (misalnya 'Child', 'Adult', 'Elder') diubah menjadi kolom-kolom baru dengan nilai biner (0 atau 1) yang menunjukkan keanggotaan suatu data dalam setiap kategori usia.
Prefix 'AgeCategory_' digunakan untuk memberi nama pada kolom baru yang dihasilkan dari one-hot encoding.

Sama seperti pada kolom 'AgeCategory', kolom 'Gender' yang berisi jenis kelamin ('Male' atau 'Female') diubah menjadi kolom-kolom baru dengan nilai biner yang menunjukkan jenis kelamin responden.
Prefix 'Gender_' digunakan untuk memberi nama pada kolom baru yang dihasilkan dari one-hot encoding.

###**Encoding**

In [51]:
df = pd.get_dummies(df)

###**Pengecekkan Hasil**

In [52]:
df.head()

,Age,Years of Experience,Salary,JobCategory,AgeCategory,AgeCategory_1,Gender_Female,Gender_Male,JobCategory_0,JobCategory_1,...,Job Title_Supply Chain Manager,Job Title_Technical Recruiter,Job Title_Technical Support Specialist,Job Title_Technical Writer,Job Title_Training Specialist,Job Title_UX Designer,Job Title_UX Researcher,Job Title_VP of Finance,Job Title_VP of Operations,Job Title_Web Developer
0,32.0,5.0,90000.0,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,28.0,3.0,65000.0,0,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,45.0,15.0,150000.0,1,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,36.0,7.0,60000.0,5,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,52.0,20.0,200000.0,1,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


Setelah kedua kolom tersebut di-transformasikan menggunakan one-hot encoding, hasilnya adalah dataframe baru dengan tambahan kolom-kolom biner yang merepresentasikan kategori usia dan jenis kelamin dari setiap responden. Hal ini mempermudah analisis dan pemodelan data, terutama ketika menggunakan algoritma pembelajaran mesin yang memerlukan fitur-fitur dalam bentuk biner.

In [53]:
df.to_csv('/content/drive/My Drive/Data Cleaned.csv', index=False)

OSError: Cannot save file into a non-existent directory: '/content/drive/My Drive'